<a href="https://colab.research.google.com/github/19rehan/Satellite-Image-to-Map-Generator/blob/main/Satellite_to_Map_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def preprocess(mask,image):

  image = tf.cast(image, tf.float32) /255.0

  return image, mask

In [4]:
#DATA AUGMENTATION LAYER

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [11]:
# 1. Base Model (Transfer Learning)
# Humne MobileNetV2 liya lekin uska 'top' (Dense layers) uda diya
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# 2. Build the Model
inputs = layers.Input(shape=[128, 128, 3])
# Apply Augmentation layers directly
x = layers.RandomFlip("horizontal_and_vertical")(inputs) # Apply RandomFlip
x = layers.RandomRotation(0.2)(x) # Apply RandomRotation
x = base_model(x) # Features extract karo (Pre-trained power!)

# Ab image ka size chota ho gaya hai (e.g., 4x4).
# 3. Transpose Convolution (Up-sampling) - To make the mask
x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu')(x) # 8x8
x = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')(x) # 16x16
# ... isi tarah layers add karte jayenge jab tak 128x128 wapas na aa jaye

# Final Layer: Output mask (1 channel for black/white mask)
outputs = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = models.Model(inputs=inputs, outputs=outputs)

In [9]:
#PRE-TRAINED MODEL ARCHITECTURE (MOBILENETV2)

base_model = tf.keras.applications.MobileNetV2(input_shape=[128,128,3], include_top=False)

inputs =  layers.Input(shape=[128,128,3])

x = data_augmentation(inputs)
x = base_model(x)

# NOW APPLY TRANSPOSE CONVOLUTION OR (UP-SAMPLING) To make the mask.

x = layers.Conv2DTranspose(64,(3,3), strides = 2, padding = 'same' , activation = 'relu')(x)
x = layers.Conv2DTranspose(32,(3,3), strides = 2, padding = 'same' , activation = 'relu')(x)


outputs = layers.Conv2D(1,(3,3), activation = 'sigmoid', padding = 'same')(x)

model = models.Model(inputs = inputs, outputs = outputs)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "functional" is incompatible with the layer: expected shape=(None, 12, 8, 128, 3), found shape=(None, 128, 128, 3)[0m

Arguments received by Sequential.call():
  • args=('<KerasTensor shape=(None, 128, 128, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_313>',)
  • kwargs={'mask': 'None'}